Kaushal Jani


**Practical** **7**

In [1]:
import sklearn
from sklearn import datasets,metrics
from sklearn.naive_bayes import GaussianNB
import numpy as np

In [6]:
X,Y = datasets.load_iris(return_X_y=True)
xtrain = X[range(0,150,2),:]
ytrain = Y[range(0,150,2)]

xtest  = X[range(1,150,2),:]
ytest  = Y[range(1,150,2)]

gclf = GaussianNB()

gclf.fit(xtrain,ytrain)
print("example of each class ",gclf.class_count_)
print("mean for given feature value for given class\n ",gclf.theta_)
print("standarad deviation for given feature value by given class\n",gclf.sigma_)
pred= gclf.predict(xtest)

print("\naccuracy ",metrics.accuracy_score(ytest,pred))


example of each class  [25. 25. 25.]
mean for given feature value for given class
  [[5.024 3.48  1.456 0.228]
 [5.992 2.776 4.308 1.352]
 [6.504 2.936 5.564 2.076]]
standarad deviation for given feature value by given class
 [[0.146624 0.1016   0.040864 0.006016]
 [0.296736 0.108224 0.220736 0.036096]
 [0.349184 0.079104 0.285504 0.074624]]

accuracy  0.96


In [10]:
import math
def gauss(mean,std,val):
   k= 1/(std* math.sqrt(2*math.pi))
   f=math.exp(-0.5*((val-mean)/std)**2)
   return k*f
def custom_fit(xtrain,ytrain,unique_v):
  mean=np.zeros((len(unique_v),xtrain.shape[1]))
  std=np.zeros((len(unique_v),xtrain.shape[1]))
  prior=[]
  for i in unique_v:                             # loop till number of class than create mask of index to where value = class than find mean and std in of each feature given class
    mask= ytrain[:] == i
    prior.append(mask.sum()/ytrain.shape)
    for j in range(0,xtrain.shape[1]):
      mean[i][j]=np.mean(xtrain[mask,j])
      std[i][j]=np.std(xtrain[mask,j])
  
  return prior,mean,std





prior,mean,std=custom_fit(xtrain,ytrain,[0,1,2])
print("prior prob \n",prior)
print("mean value for each feature given class\n",mean)
print("standard deviation  for each feature given class\n",std)

def custom_predict(xtest):
  prior,mean,std=custom_fit(xtrain,ytrain,[0,1,2])
  post=[]
  pre=[]
  for i in xtest:       # it extracts row from testing set
    post=[]
    for k in range(0,len(prior)):  # we have all std and mean for all feature given class(3 * 4) matrics 
      x=1
      for j in range(0,len(i)):   # now extract each element from row and find its likely hood and multiply all likely hood as per formula 
        x=x*gauss(mean[k][j],std[k][j],i[j])
      post.append(x)              # store result given class so 1th entry corrospends to 1st class 
      result=[]
    for i in range(0,len(prior)):
      result.append(prior[i] * post[i])  # multiply prior and likelhood
   # print(result)
    pre.append(result.index(max(result))) # find max prob of result and assign its index as class becuse in post we store each class's prob like this
  return pre



pre= custom_predict(xtest)
print("predictions are \n",pre)
print("accuracy ",metrics.accuracy_score(ytest,pre))

prior prob 
 [array([0.33333333]), array([0.33333333]), array([0.33333333])]
mean value for each feature given class
 [[5.024 3.48  1.456 0.228]
 [5.992 2.776 4.308 1.352]
 [6.504 2.936 5.564 2.076]]
standard deviation  for each feature given class
 [[0.38291513 0.31874755 0.20214846 0.07756288]
 [0.5447348  0.32897416 0.4698255  0.18998947]
 [0.59091793 0.28125433 0.53432574 0.27317394]]
predictions are 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2]
accuracy  0.96


Mutinomial Classifier

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [14]:
"""Here 24 datastring in data 0to 19 for trianing dataset in which 0 to 9 for cricket 10 to 19 for footbal and 20 to 23 for testing """


data=["Cricket is a bat-and-ball game played between two teams of eleven players on a field at the centre of which is a 22-yard pitch with a wicket at each end, each comprising two bails balanced on three stumps.",
      "Cricket was invented in the vast fields of England, supposedly by shepherds who herded their flock. Later on this game was shown benevolence by aristocrats, and now has the stature of being England's national game. After a century now, cricket stands in the international arena, with a place of its own.",
      "Bowlers can come in many different varieties and this video explains all of the different types of bowling style. Swing bowlers, seamers and spinners tactics and techniques are all shown off, as well as how they get their crucial wickets and dot balls.",
      "Cricket is a game played with a bat and ball on a large field, known as a ground, between two teams of 11 players each.",
      "The batting team must score as many ‘runs’ as possible, by hitting the ball and running to the other end of the pitch. If the batsman can reach the other end of the pitch successfully, he scores 1 ‘run’. If he can reach the other end of the pitch and return, he scores 2 runs etc.If he hits the ball to the edge of the field, he scores 4 runs. If he can hit the ball to the edge of the field without bouncing, he scores 6 runs.",
      "The earliest reference to cricket is in South East England in the mid-16th century. It spread globally with the expansion of the British Empire, with the first international matches in the second half of the 19th century. The game's governing body is the International Cricket Council",
      "There are two batsman up at a time, and the batsman being bowled to (the striker) tries to hit the ball away from the wicket. ",
      "A hit may be defensive or offensive. A defensive hit may protect the wicket but leave the batsmen no time to run to the opposite wicket. In that case the batsmen need not run, and play will resume with another bowl.",
      "A ball hit to or beyond the boundary scores four points if it hits the ground and then reaches the boundary, six points if it reaches the boundary from the air (a fly ball).",
      "The earliest reference to an 11-a-side match, played in Sussex for a stake of 50 guineas, dates from 1697. In 1709 Kent met Surrey in the first recorded intercounty match at Dartford, and it is probable that about this time a code of laws (rules) existed for the conduct of the game, although the earliest known version of such rules is dated 1744. ",
      "Football is a family of team sports that involve, to varying degrees, kicking a ball to score a goal. Unqualified, the word football normally means the form of football that is the most popular where the word is used.",
      "Football, also called association football or soccer, game in which two teams of 11 players, using any part of their bodies except their hands and arms, try to maneuver the ball into the opposing team’s goal. Only the goalkeeper is permitted to handle the ball and may do so only within the penalty area surrounding the goal. ",
      "Football is the world’s most popular ball game in numbers of participants and spectators. Simple in its principal rules and essential equipment, the sport can be played almost anywhere, from official football playing fields (pitches) to gymnasiums, streets, school playgrounds, parks, or beaches.",
      "Modern football originated in Britain in the 19th century. Since before medieval times, “folk football” games had been played in towns and villages according to local customs and with a minimum of rules. Industrialization and urbanization, which reduced the amount of leisure time and space available to the working class, combined with a history of legal prohibitions against particularly violent and destructive forms of folk football to undermine the game’s status from the early 19th century onward. ",
      "In 1863 a series of meetings involving clubs from metropolitan London and surrounding counties produced the printed rules of football, which prohibited the carrying of the ball. Thus, the “handling” game of rugby remained outside the newly formed Football Association (FA). Indeed, by 1870 all handling of the ball except by the goalkeeper was prohibited by the FA.",
      "The game of US football evolved in the 19th century as a combination of rugby and soccer. The first intercollegiate match was played in 1869 between Princeton University and Rutgers College. In 1873, the first collegiate rules were standardized and the Ivy League was formed. Collegiate football grew into one of the most popular American sports. Professional football began in the 1890s, but did not become a major sport until after World War II. The National Football League (NFL) was formed (from an earlier association) in 1922; in 1966 it subsumed the rival American Football League (created in 1959). The NFL is now divided into an American and a National conference; the conference winners compete for the Super Bowl championship. A Football Hall of Fame is located in Canton, Ohio.",
      "A match consists of two 45 minute periods known as the first and second half. In some instances, extra time can be played, two 15 minute periods. If side remain level after extra time a replay or penalty kicks can occur. Each team will take 5 penalties each, if scores remain level after both teams take their allotted 5 penalties then sudden death occurs. The first team to miss their penalty will lose if the opponents have scored their sudden death penalty kick. Extra time and penalties tend to only occur in tournaments and cup competitions. League games will result in a draw if both teams score the same amount of goals during the 90 minutes.",
      "Each player will wear football boots, studs, blades or moulded this is depended on the pitch conditions.Shin pads must be worn by all players, to protect their shins. Some players ware ankle pads, however these are not compulsory.Goal keepers are allowed to wear gloves.",
      "Yellow card – a yellow card is awarded by the referee for a breach of the rules that he does not consider serious or dangerous. A player who receives a yellow card is known as getting “booked” as the referee records the players name in his book.",
      "Substitution – 11 players are allowed on the pitch unless a player has been sent off or a team has made the maximum of 3 substitutions. 3 substitutions are allowed out of 5 out field players and a sub goalkeeper. You are not allowed to sub a player who is in the course of being sent off or already dismissed.",
      "Umpires have a key role in the game as they monitor the proceedings. They decide whether the batsman is out, decide on no-ball, wide, and ensure both teams are playing according to the rules.",
      "If the scores are level after 90 minutes then the game will end as a draw apart from in cup games where the game can go to extra time and even a penalty shootout to decide the winner.",
      "In some instances, on-field umpires find it tough to give few decisions like boundaries, out, no-ball, etc. Therefore, they seek help of another umpire, called third-umpire.",
      "The game of football takes its form. The most admitted story tells that the game was developed in England in the 12th century. "]


testdata=[]

count=0
for i in data:
  count=count+len(i)
print("total letters in data",count)
data_vector=CountVectorizer()
vdata=data_vector.fit_transform(data) # converts the data into vector representation
traindata=vdata[:-4]                   # first  20 for training 
traindata=traindata.toarray()
print("no of feature(key words) of given dataset",len(data_vector.get_feature_names()))

testdata=vdata[-4:].toarray()         # last 4 for testing

print(traindata)
answer=["Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Football","Football","Football","Football","Football","Football","Football","Football","Football","Football"]
testans=["Cricket","Football","Cricket","Football"]
clf=MultinomialNB()
clf.fit(traindata,answer)
print(clf.predict(testdata))
print("accuracy ",metrics.accuracy_score(testans,clf.predict(testdata)))

total letters in data 7097
no of feature(key words) of given dataset 490
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 3 0]
 [1 0 0 ... 0 0 1]]
['Cricket' 'Football' 'Cricket' 'Football']
accuracy  1.0


Multivariate Bernoulli Classifier

In [13]:
data_vector=CountVectorizer(binary=True)
vdata=data_vector.fit_transform(data) 
traindata=vdata[:-4]                   # first  20 for training 
traindata=traindata.toarray()
print("Number  of feature(key words ) of given dataset are ",len(data_vector.get_feature_names()))

testdata=vdata[-4:].toarray()         # last 4 for testing

print(traindata)
answer=["Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Cricket","Football","Football","Football","Football","Football","Football","Football","Football","Football","Football"]
testans=["Cricket","Football","Cricket","Football"]
clf=BernoulliNB()
clf.fit(traindata,answer)
print(clf.predict(testdata))
print("\naccuracy ",metrics.accuracy_score(testans,clf.predict(testdata)))

Number  of feature(key words ) of given dataset are  490
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [1 0 0 ... 0 0 1]]
['Cricket' 'Cricket' 'Cricket' 'Cricket']

accuracy  0.5
